In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
hyperparameters = {
    'vocab_size': 10,
    'embed_size': 128,
    'hidden_size': 256,
    'num_layers': 2,
    'lr': 1e-3,
    'batch_size': 64,
    'val_split': 0.2,
    'test_split': 0.2,
    'num_workers': 4,
    'max_epochs': 10,
    'warmup_steps': 1000,
}

In [ ]:
import pandas as pd
from dataset import SudokuDataModule

data = pd.read_csv('data/sudoku_small.csv')
dataset = SudokuDataModule(data, hyperparameters)

In [ ]:
from model import SudokuSolver
from lightning.pytorch.utilities.model_summary import ModelSummary

model = SudokuSolver(hyperparameters)
ModelSummary(model, max_depth=1)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.tuner.tuning import Tuner
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor

trainer = Trainer(
    max_epochs=10,
    logger=TensorBoardLogger('logs', name='sudoku-solver', log_graph=True),
    callbacks=[
        ModelCheckpoint(monitor='val_loss', save_top_k=1, mode='min'),
        EarlyStopping(monitor='val_loss', patience=5, mode='min'),
        LearningRateMonitor(logging_interval='step')
    ],
    enable_checkpointing=True,
    enable_progress_bar=True,
)
tuner = Tuner(trainer)

In [ ]:
tuner.lr_find(model, datamodule=dataset, min_lr=1e-10, max_lr=10, num_training=1000)

In [ ]:
trainer.fit(model, datamodule=dataset)

In [ ]:
trainer.test(model, datamodule=dataset)